# Tutorial 2: Understanding Traces

The most fundamental concept in ToolBrain's learning process is the **Trace**. A Trace is a complete, high-fidelity record of an agent's attempt to solve a task. Understanding its structure is key to debugging, creating custom reward functions, and analyzing agent behavior.

This tutorial explores the `Trace` and `Turn` data structures, defined in `toolbrain/core_types.py`.

## The Structure of a Trace

A `Trace` is simply a list of `Turn` objects:

```python
Trace = List[Turn]
```

Each `Turn` represents a single step or interaction in the agent's execution flow. By stringing these turns together, the `Trace` provides a complete, chronological history of the agent's reasoning process.

## The Anatomy of a Turn

A `Turn` is a `TypedDict`, a dictionary with a defined structure. It contains the following fields, each capturing a critical piece of information:

```python
class Turn(TypedDict):
    prompt_for_model: str
    model_completion: str
    parsed_completion: ParsedCompletion
    tool_output: Optional[str]
    action_output: Optional[str]   
    formatted_conversation: Optional[str] 

```

Let's break down each field:

1.  **`prompt_for_model: str`**
    - **What it is**: The exact, complete string of text that was fed into the language model at the beginning of the turn.
    - **Why it's important**: This is the full context the LLM had. It includes the initial user query, the system prompt, and the history of all previous turns (thoughts, tool calls, and tool outputs). For RL, it is important to know precisely what the model *saw*.

2.  **`model_completion: str`**
    - **What it is**: The raw, unedited string of text generated by the language model in response to the `prompt_for_model`.
    - **Why it's important**: This is the model's pure output, before any parsing or processing. It's the ground truth of what the model *did*. This is what gets compared against during the learning process.

3.  **`parsed_completion: ParsedCompletion`**
    - **What it is**: A dictionary that contains the structured breakdown of the `model_completion`. The `CodeAgent` parses the raw output into distinct components:
        - **`thought`**: The agent's reasoning or plan.
        - **`tool_code`**: The Python code the agent decided to execute.
        - **`final_answer`**: The final answer provided by the agent.
    - **Why it's important**: This structured data makes it easy to programmatically access the agent's actions and decisions without needing to parse the raw text yourself.

4.  **`tool_output: Optional[str]`**
    - **What it is**: A string containing the result of executing the `tool_code`. This could be the return value of a function (e.g., `'12'`) or an error message if something went wrong (e.g., `'TypeError: add() missing 1 required positional argument: 'b''`). It is `None` if no tool code was executed in the turn.
    - **Why it's important**: This represents the feedback from the environment. The `tool_output` from the current turn is appended to the conversation history and becomes part of the `prompt_for_model` for the *next* turn, allowing the agent to react to the results of its actions.


5.  **`action_output: Optional[str]`**
    - **What it is**: The result from executing the tool code (if any)
.
    - **Why it's important**: Optional data which provides additional context about the agent execution. 

6.  **`formatted_conversation: Optional[str]`**
    - **What it is**: The result from executing the tool code (if any)
.
    - **Why it's important**: Optional data for additional context, provided by the agent.
## Example: A Single Turn in a Trace

Imagine our agent is asked to calculate `5 + 7`. The first `Turn` in the `Trace` might look something like this (simplified for clarity):

```json
{
    "prompt_for_model": "<|SYSTEM|>You are a helpful assistant...<|USER|>Use the add tool to calculate 5 + 7",
    "model_completion": "Thought: To solve this task, I need to perform the addition operation using the add tool.\n<code>\nadd_result = add(5, 7)\nprint(add_result)\n</code>",
    "parsed_completion": {
        "thought": "To solve this task, I need to perform the addition operation using the add tool.\n<code>\nadd_result = add(5, 7)\nprint(add_result)\n</code>",
        "tool_code": "add_result = add(5, 7)\nprint(add_result)",
        "final_answer": null
    },
    "tool_output": "Execution logs:\n12\nLast output from code snippet:\nNone",
    "action_output": "",
}
```

In the next turn, the `tool_output` ("12") would be added to the history, and the agent would likely produce a `final_answer`.

---

By preserving this detailed, step-by-step information, the `Trace` and `Turn` objects provide the data for both calculating rewards and fine-tuning the agent's underlying model. In the next tutorial, we will see how to use this data to create custom reward functions.